In [ ]:
import sys
from pathlib import Path
pp = str(Path('.').absolute().parent)
if pp not in sys.path:
    sys.path.append(pp)

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt 
import networkx as nx
import numpy as np

from qpr.vtr_utils import parse_rr_graph_xml, pos_rr_graph, draw_rr_graph

%matplotlib inline

In [ ]:
np.random.seed(1234)

fpath = Path().absolute().parent / 'sharedgraphstuff' / 'rrgraph.xml'
graph, node_dict, grid_dict, clock_nodes = parse_rr_graph_xml(fpath)
display(f'num edges: {len(graph.edges)}, num nodes: {len(graph.nodes)}')

In [ ]:
%%time
# %matplotlib notebook
# fig, ax = plt.subplots(figsize=(10, 10))
fig, ax = plt.subplots(figsize=(25, 25))

# dx, dy from the center of the tile to get to the start of the bbox edge, and
# laying out the pins in order clockwise around the bounding box of the (sub?)block
# step_dx, step_dy, which way to move along the edge to place the pins
side_deltas = dict(
    RIGHT=np.array([1, 1, 0, -1]), BOTTOM=np.array([1, -1, -1, 0]),
    LEFT=np.array([-1, -1, 0, 1]), TOP=np.array([-1, 1, 1, 0])
)
# IOPIN, create concentric rings to avoid overlap
# SINK/SOURCE linear set of nodes just offset vertically
fudge_factor = dict(SINK=.1, SOURCE=0, IPIN=.1, OPIN=0)
# w, h, channel bounding box size
# orig_offset_x, orig_offset_y, origin of the coordinate system for channel
# location is offset relative the coordinate system for blocks
chan_bbox = dict(CHANX=(1, .1, 0, 1), CHANY=(.1, 1, 1, 0))
# the edges of the bounding box of a block is this much away from its center
bbb_half_edge = .2
# put the route node (SINK/SOURCE) bouding boxs in side of a smaller bounding
# box inside of the block bounding box
rnbb_half_edge = .15

pos = pos_rr_graph(node_dict, grid_dict, side_deltas, fudge_factor, chan_bbox, bbb_half_edge)


node_cols = dict(
    SOURCE='r', SINK='tab:blue', CHANX='y', CHANY='y', IPIN='g', OPIN='m'
)
draw_subgraph = graph.subgraph(list(pos.keys()))
for i in node_cols:
    draw_nodes = [j for j in node_dict if node_dict[j]['type'] == i]
    nx.draw_networkx(
        draw_subgraph, nodelist=draw_nodes, edgelist=[], pos=pos,
        node_color=node_cols[i],
    )
draw_nodes = [i for i in node_dict if node_dict[i]['is_clock']]
nx.draw_networkx(
    draw_subgraph, nodelist=draw_nodes, edgelist=[], pos=pos,
    node_color='tab:brown',
)
    
nx.draw_networkx_edges(draw_subgraph, pos=pos)
# nx.draw_networkx(draw_subgraph, pos=pos, ax=ax)
ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
ax.grid(True)
fig.tight_layout()